## Scenario 1 -  Single collection RAG

### **SupportPatterns** - Support Training & Education Platform

- Develops training materials and courses for customer support professionals
- Uses aggregated, anonymized support conversations to create realistic training scenarios

### Solution

Collect as much conversation data between support agents and customers as possible. 

Analyse this data to identify common patterns and develop training materials based on these patterns.

### Helper functions for downloads

In [1]:
from pathlib import Path
from typing import Literal


def download_datafiles(setup: Literal["ollama", "cohere"]):
    filepaths_set = {
        "ollama": (
            "https://weaviate-workshops.s3.eu-west-2.amazonaws.com/odsc-europe-2024/twitter_customer_support_weaviate_export_50000_nomic.h5",
            Path("data/twitter_customer_support_nomic.h5")
        ),
        "cohere": (
            "https://weaviate-workshops.s3.eu-west-2.amazonaws.com/odsc-europe-2024/twitter_customer_support_weaviate_export_50000_cohere-embed-multilingual-light-v3.0.h5",
            Path("data/twitter_customer_support_cohere.h5"),
        )
    }

    filepaths = filepaths_set[setup]

    if not filepaths[1].exists():
        print(f"Downloading {filepaths[0]}")
        filepaths[1].parent.mkdir(parents=True, exist_ok=True)
        import urllib.request
        urllib.request.urlretrieve(filepaths[0], filepaths[1])
    else:
        print(f"File already exists: {filepaths[1]}")
    return True

## AI Models

This workshop is set up for you to work with local, Ollama models, or API-based Cohere models. Follow either [Ollama](#ollama) or [Cohere](#cohere) instructions below.


### Ollama

To use Ollama models for this workshop, uncomment the below cells and run them.

In [32]:
# !ollama pull nomic-embed-text && ollama pull gemma2:2b

In [33]:
# download_datafiles("ollama")

# model_type = "ollama"

### Cohere 

To use the Cohere API for this workshop, run the below code cell to configure the variables:

In [4]:
download_datafiles("cohere")

model_type = "cohere"

File already exists: data/twitter_customer_support_cohere.h5



### Create the collection


In [5]:
from weaviate.classes.config import Configure

if model_type == "ollama":
    vectorizer_config = Configure.NamedVectors.text2vec_ollama(
        name="text_with_metadata",
        source_properties=["text", "company_author"],
        vector_index_config=Configure.VectorIndex.hnsw(),
        api_endpoint="http://host.docker.internal:11434",
        model="nomic-embed-text",
    )
    generative_config = Configure.Generative.ollama(
        api_endpoint="http://host.docker.internal:11434",
        model="gemma2:2b"
    )
else:
    vectorizer_config = Configure.NamedVectors.text2vec_cohere(
        name="text_with_metadata",
        source_properties=["text", "company_author"],
        vector_index_config=Configure.VectorIndex.hnsw(),
        model="embed-multilingual-light-v3.0",
    )

    generative_config = Configure.Generative.cohere(
        model="command-r-plus"
    )


In [6]:
import os
import weaviate
from weaviate.classes.config import Property, DataType, Configure

try:
    cohere_api_key = os.environ["COHERE_API_KEY"]
except:
    print("Warning - Cohere API key not yet - if using Cohere, please set the COHERE_API_KEY environment variable")
    cohere_api_key = ""

client = weaviate.connect_to_local(
    headers={"X-Cohere-Api-Key": cohere_api_key}
)

collection_name = "SupportChat"

# For re-running the demo only: Delete existing collection if it exists
client.collections.delete(collection_name)

# Create a new collection with specified properties and vectorizer configuration
chunks = client.collections.create(
    name=collection_name,
    properties=[
        Property(name="text", data_type=DataType.TEXT),
        Property(name="dialogue_id", data_type=DataType.INT),
        Property(name="company_author", data_type=DataType.TEXT),
        Property(name="created_at", data_type=DataType.DATE),
    ],
    vectorizer_config=[vectorizer_config],
    generative_config=generative_config,
)

### Helper functions for loading data

In [7]:
import h5py
import json
import numpy as np
from typing import Literal
from pathlib import Path


def get_hdf5_obj(file_path):
    with h5py.File(file_path, "r") as hf:
        for uuid in hf.keys():
            src_obj = hf[uuid]

            # Get the object properties
            properties = json.loads(src_obj["object"][()])

            # Get the vector(s)
            vectors = {}
            for key in src_obj.keys():
                if key.startswith("vector_"):
                    vector_name = key.split("_", 1)[1]
                    vectors[vector_name] = np.asarray(src_obj[key])

            yield uuid, properties, vectors


def get_data_obj(model_type: Literal["ollama", "cohere"]):
    file_path = Path("data/twitter_customer_support_nomic.h5")
    if model_type == "cohere":
        file_path = Path("data/twitter_customer_support_cohere.h5")

    for uuid, properties, vectors in get_hdf5_obj(file_path):
        yield uuid, properties, vectors

### Load data

In [8]:
from tqdm import tqdm

with client.batch.fixed_size(batch_size=200) as batch:
    for uuid, properties, vectors in tqdm(get_data_obj(model_type)):
        batch.add_object(
            collection=collection_name,
            uuid=uuid,
            properties=properties,
            vector={"text_with_metadata": vectors["text_with_metadata"]},
        )

50000it [00:20, 2444.92it/s]


In [9]:
print(f"Processed {len(client.batch.results.objs.all_responses)} objects.")

Processed 50000 objects.


/Users/jphwang/code/demo-code/202501-vectordb-patterns/.venv/lib/python3.10/site-packages/weaviate/warnings.py:213: DeprecationWarning: Dep020: The `all_responses` attribute in the `BatchResults` object is deprecated and will be removed by Q4 2024. Please instead use the `errors` and `uuids` attributes.
  warnings.warn(


In [10]:
if len(client.batch.failed_objects) > 0:
    print("*" * 80)
    print(f"***** Failed to add {len(client.batch.failed_objects)} objects *****")
    print("*" * 80)
    print(client.batch.failed_objects[:3])

### Confirm data load

In [11]:
support_chats = client.collections.get(collection_name)

In [12]:
response = support_chats.query.fetch_objects(limit=2, include_vector=True)

In [13]:
print(response.objects[0].uuid)

00014ba3-ea82-524b-a45c-99b4153b74bc


In [14]:
for k, v in response.objects[0].properties.items():
    print(f"\n|| {k} || \n{v}")


|| created_at || 
2017-10-03 14:47:07+00:00

|| text || 
User_164303: NRC Feature Request: Support for interval training. 😀
NikeSupport: Here to help. Have you checked out the option to run a Speed run which is interval training?
User_164303: I have not. Will definitely check it out. Thanks!

|| company_author || 
NikeSupport

|| dialogue_id || 
204089


In [15]:
for k, v in response.objects[0].vector.items():
    print(k, v[:3])

text_with_metadata [-0.042388916015625, 0.041412353515625, -0.0445556640625]


### Queries

#### Helper function for displaying objects

In [16]:
def display_objects(response):
    for o in response.objects:
        print(o.uuid, "\n")
        print(o.properties["text"][:100], "\n")

In [17]:
response = support_chats.query.near_text("return process", limit=3)
display_objects(response)

31a21f1b-61aa-5c01-bcbf-6dce68b2cb5e 

User_119904: @115850 I have bought a product and now it's size is not matching I want to return it a 

3247d280-de61-515f-a388-caaac81770c8 

User_206228: please check the DM sent to @AmazonHelp abd revert.
AmazonHelp: Hi, we have responded t 

32f8710d-8ea7-5e04-a1f2-4be40ce92dab 

User_207558: Second DOA harddrive from @NeweggService Starting the return process again :/ Trying fr 



In [18]:
response = support_chats.query.bm25("return process", limit=3)
display_objects(response)

32f8710d-8ea7-5e04-a1f2-4be40ce92dab 

User_207558: Second DOA harddrive from @NeweggService Starting the return process again :/ Trying fr 

31a21f1b-61aa-5c01-bcbf-6dce68b2cb5e 

User_119904: @115850 I have bought a product and now it's size is not matching I want to return it a 

0e2bf8ee-425e-5936-b703-dd9d4494f9dc 

User_234840: Hi, I was charged $1.00 while making a return shipping label online. Why was this the c 



In [19]:
response = support_chats.query.hybrid("return process", limit=3)
display_objects(response)

31a21f1b-61aa-5c01-bcbf-6dce68b2cb5e 

User_119904: @115850 I have bought a product and now it's size is not matching I want to return it a 

32f8710d-8ea7-5e04-a1f2-4be40ce92dab 

User_207558: Second DOA harddrive from @NeweggService Starting the return process again :/ Trying fr 

3247d280-de61-515f-a388-caaac81770c8 

User_206228: please check the DM sent to @AmazonHelp abd revert.
AmazonHelp: Hi, we have responded t 



In [20]:
response = support_chats.generate.fetch_objects(
    limit=20,
    grouped_task="What patterns are we seeing here in these issues?"
)

In [21]:
print(response.generated)

Based on the provided data, there seem to be several patterns emerging from these customer issues:

- **Delivery and Shipping Issues**: Multiple users have raised concerns about late deliveries, missing packages, and issues with tracking. This suggests that delivery and shipping are areas where customers frequently encounter problems, and it might be beneficial to review and improve these processes.

- **Product or Service Quality**: Some customers have expressed dissatisfaction with the quality of products or services, such as in-flight meals, food orders, and technical issues with phone updates. Addressing these concerns could enhance customer satisfaction and loyalty.

- **Customer Service Accessibility**: A few users mentioned difficulties in reaching customer support or receiving timely responses. This indicates that there might be room for improvement in the accessibility and responsiveness of customer service channels.

- **Account and Ordering Issues**: There are instances wher

## Example use cases

- Develop training materials
    - Investigate common patterns in support conversations
    - Identify common issues and resolutions

In [22]:
response = support_chats.generate.near_text(
    query="return process",
    limit=15,
    grouped_task="Describe some common problems that customers may complain about.",
)

In [23]:
print(response.generated)

Here is a list of common problems that customers may complain about, based on the provided data:

- Issues with product size or parts missing, leading to return and refund requests.
- Difficulty in arranging returns and confusion about return shipping charges.
- Delays in receiving refunds after returning items, with customers seeking confirmation and faster processing.
- Inconvenience caused by having to print return forms and issues with not receiving call backs or updates on the return process.
- Threats of legal action if a refund is not provided promptly.
- Inability to leave negative feedback for poor service without facing consequences.
- Confusion about the procedure to obtain a refund for courier charges incurred during self-ship returns.
- Concerns about the time taken for returns to be processed and reflected in the system.
- Frustration with having to bear additional costs for returning items, especially when it is not clearly communicated beforehand.

These complaints high

In [24]:
response = support_chats.generate.near_text(
    query="return process",
    limit=15,
    grouped_task="Describe some common problems that customers may complain about, and suggest top 5 training scenaior that may be useful for new support agents.",
)

In [25]:
print(response.generated)

Common customer complaints:
- Issues with returning products, including confusion over return policies, unexpected charges, and delays in processing returns.
- Discrepancies in product sizing, resulting in the need for returns or exchanges.
- Defective or missing parts in delivered items, requiring replacements or repairs.
- Delayed refunds for returned items, with customers seeking confirmation and faster processing.
- Inability to pick up returns, leading to customer frustration and threats of legal action.

Top 5 training scenarios for new support agents:
1. Handling return requests: Practice addressing customer concerns about return policies, providing clear and accurate information, and offering alternatives like exchanges or refunds.
2. Sizing and product details: Train agents to handle queries related to product sizing, offering solutions like size guides or recommendations for similar products.
3. Managing defective or missing parts: Simulate scenarios where customers receive i

### Resource management

- How much memory are we using?
- How will this scale with more data?

## When to use this pattern

- Is any of the data isolated from the others?
- What use cases might not be covered by this architecture?


## Demo application

- Outside of the notebook
